In [2]:
import pickle
import numpy as np

In [2]:
import transformers as ppb

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
questions = pickle.load(open("questions", "rb"))
questions = list(questions)

In [4]:
while questions.index(''): 
    questions.pop(questions.index(''))

ValueError: '' is not in list

In [5]:
len(questions)

33543

In [7]:
from semantic_text_similarity.models import WebBertSimilarity

web_model = WebBertSimilarity(device='cuda', batch_size=1024) #defaults to CPU prediction


  8%|▊         | 32168960/405359924 [39:22<3:56:59, 26245.57B/s]

Failed to download model: web-bert-similarity


KeyboardInterrupt: 

In [ ]:
tuples = list(product(query,questions))
response = []
predictions = web_model.predict(tuples)

In [7]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [8]:
from bert_serving.client import BertClient
bc = BertClient(port=5555, port_out=5556)

In [9]:
doc_vecs = bc.encode(questions[:200])

/Users/felipe/opt/miniconda3/envs/bert/lib/python3.6/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


In [10]:
len(doc_vecs)

200

In [11]:
with open('doc_vecs', "bw") as f:
        pickle.dump(doc_vecs, f)

In [83]:
query = 'cadastro no Identity com o endereço de e-mail incorreto.'

In [84]:
query_vec = bc.encode([query])[0]

In [85]:
# compute normalized dot product as score
score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
topk_idx = np.argsort(score)[::-1]

In [86]:
topk_idx

array([195,   4, 125, 192,   5, 118, 175,  87,  24,  93,  64, 128,  47,
       126, 117,  65, 177,  51,  34,  41,  57, 111,   2,  67,  56,  29,
        12,  76,  35, 131, 157,  61, 185, 108,  45, 190, 150, 179, 141,
        91,  66,  85, 193,  59,  37,  25, 115,  88, 174, 167,  31, 171,
       164,  80,  49, 104, 188, 176,  50, 113, 196,  21,  83, 152,  82,
       146, 144,  11,  99, 154, 123,  28, 194, 133, 143, 138,  89, 100,
       147,   7,   1, 155, 122, 114, 163,  27,  86,  16, 149,  26, 158,
        70,  30, 168,  98, 130,  78,  62,  71, 183,  69, 148,  81, 198,
       184,  54,  48,  72,  32, 101,  36,  55, 199,  42,  73, 110,  14,
       151,  63,  84,  79,  15, 135,  33, 139, 180, 197, 170,  58,  53,
       186, 109,   9,  44,  94, 121,  13,  22,  10, 189,   8, 166,  43,
         0, 191, 134, 173,  52, 112, 161, 106,  75,  92,  19, 127, 136,
        39,  96,  38, 145,   3, 142,  18,  40, 140,  90,  17, 120,  97,
       182, 153,  20,  95, 160, 124, 137, 102, 169, 178, 116, 16

In [87]:
questions[4]

'Um dos nossos usuários está cadastrado no fluig Identity e no fluig com o endereço de e-mail incorreto.Como faço para corrigir o endereço de e-mail do usuário nos dois sistemas?'

In [88]:
questions[195]

'Após configurar o servidor do fluig para acessar uma conta do Gmail para envio de e-mails, os mesmos não são enviados apresentando a mensagem de erro no log "534 5.7.14" Falha na autenticação.'

#### Importing the pre trained model

In [13]:
import warnings
warnings.filterwarnings('ignore')

#model_class, tokenizer_class, pretrained_weights = (ppb.BertForSequenceClassification, ppb.BertTokenizer, 'bert-base-multilingual-cased')
tokenizer_class, pretrained_weights = (ppb.BertTokenizer, 'bert-base-multilingual-cased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
#model = model_class.from_pretrained(pretrained_weights,
#                                    num_labels = 9, # integer, default 2. Number of classes to use when the model is a classification model (sequences/tokens)
#                                    output_attentions = False, # Should the model returns attentions weights.
#                                    output_hidden_states = False,  # Should the model returns all hidden-states.
#                                   )

#### Tokenization

In [14]:
tokenized = [tokenizer.encode(x, add_special_tokens=True) for x in questions[:5]]

In [16]:
# Print the original sentence.
print(' Original: ', questions[1])

# Print the sentence mapped to token ids.
print('Token IDs: ', ' '.join(tokenizer.convert_ids_to_tokens(tokenized[1])))

# Print the sentence split into tokens.
print('Tokenized: ', tokenized[1])

 Original:  Ao tentar alterar as informações de um usuário e bloquear o mesmo, uma mensagem é apresentada: result returns more than one elements. Por que essa mensagem aparece?
Token IDs:  [CLS] Ao tentar alter ##ar as informações de um usu ##ário e bloque ##ar o mesmo , uma mens ##agem é apresenta ##da : result returns more than one elements . Por que essa mens ##agem aparece ? [SEP]
Tokenized:  [101, 17607, 110694, 42141, 10354, 10146, 68263, 10104, 10293, 44224, 15211, 173, 99406, 10354, 183, 13544, 117, 10437, 12380, 29111, 263, 49320, 10229, 131, 14493, 38302, 10798, 11084, 10464, 17464, 119, 12399, 10121, 21553, 12380, 29111, 20962, 136, 102]


#### Padding

Pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [20]:
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized])

In [21]:
np.array(padded).shape

(5, 243)

In [22]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5, 243)

In [23]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

NameError: name 'torch' is not defined

In [48]:
input_ids.shape

torch.Size([70159, 512])

In [49]:
attention_mask.shape

torch.Size([70159, 512])

In [ ]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()